<a href="https://colab.research.google.com/github/martin-lopez25/brecha-app/blob/main/TP2/Brechasapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade dash pandas plotly jupyter-dash openpyxl #instalara dash para app

In [3]:
import pandas as pd
import numpy as np
import re
from math import ceil
import os
from pathlib import Path
import unicodedata
import plotly.express as px
from io import BytesIO
import base64
from dash import dcc, html, Input, Output, dash_table, State
from jupyter_dash import JupyterDash

In [5]:
ideal=pd.read_csv('/content/drive/MyDrive/imms bienestar/plantillas_ideales_minimas_24_04_2025.csv')
print(ideal.shape)
real=pd.read_csv('/content/drive/MyDrive/imms bienestar/nomina_general_2_6_2025_9_00am (1).csv')
print(real.shape)


(337322, 12)
(300510, 12)


In [6]:
ideal.head()

,entidad,clues_imb,nombre_de_la_unidad,nivel_atencion,codigo_cnpm,denominacion_del_puesto,mat,ves,noc_a,noc_b,ja,total
0,BAJA CALIFORNIA,BCIMB000080,RUIZ Y 14,PRIMER NIVEL,CG014,ADMINISTRADOR CENTRO DE SALUD,0,0,0,0,0,0
1,BAJA CALIFORNIA,BCIMB000080,RUIZ Y 14,PRIMER NIVEL,OP006,AUXILAR DE INTENDENCIA,2,2,1,1,1,7
2,BAJA CALIFORNIA,BCIMB000080,RUIZ Y 14,PRIMER NIVEL,OP003,AUXILIAR ADMINISTRATIVO DE ARCHIVO CLÍNICO,1,1,0,0,1,3
3,BAJA CALIFORNIA,BCIMB000080,RUIZ Y 14,PRIMER NIVEL,OP001,AUXILIAR DE ADMINISTRACIÓN,1,1,0,0,0,2
4,BAJA CALIFORNIA,BCIMB000080,RUIZ Y 14,PRIMER NIVEL,OP002,AUXILIAR DE ALMACÉN,1,0,0,0,0,1


In [8]:
ideal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337322 entries, 0 to 337321
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   entidad                  337322 non-null  object
 1   clues_imb                337322 non-null  object
 2   nombre_de_la_unidad      337322 non-null  object
 3   nivel_atencion           337322 non-null  object
 4   codigo_cnpm              337322 non-null  object
 5   denominacion_del_puesto  332967 non-null  object
 6   mat                      337322 non-null  int64 
 7   ves                      337322 non-null  int64 
 8   noc_a                    337322 non-null  int64 
 9   noc_b                    337322 non-null  int64 
 10  ja                       337322 non-null  int64 
 11  total                    337322 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 30.9+ MB


In [9]:
ideal = ideal.dropna(subset=['clues_imb', 'codigo_cnpm'])
numeric_cols = ['total', 'mat', 'ves', 'noc_a', 'noc_b', 'ja']
ideal[numeric_cols] = ideal[numeric_cols].apply(lambda x: np.ceil(x.astype(float)))
count_combinations = ideal.groupby(['clues_imb', 'codigo_cnpm']).size().reset_index(name='n')
duplicates = count_combinations[count_combinations['n'] > 1]
# Obtener valores únicos (equivalente a unique)
vec_ideal = ideal['clues_imb'].unique()  # Aritmética - Ideal
ideal_cnpm = ideal[~ideal['codigo_cnpm'].isna()].groupby(
    ['clues_imb', 'codigo_cnpm'], as_index=False)['total'].sum().rename(
    columns={'total': 'total_ideal'})


In [10]:
print(real.shape)
real.head()
real.info()

(300510, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300510 entries, 0 to 300509
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   ID TRABAJADOR           300510 non-null  int64 
 1   NOMBRE COMPLETO         300510 non-null  object
 2   CURP                    300510 non-null  object
 3   RFC                     300510 non-null  object
 4   CLUES                   300510 non-null  object
 5   PUESTO                  300510 non-null  object
 6   ESTATUS                 300510 non-null  object
 7   UNIDAD DE COMISIÓN      300158 non-null  object
 8   CÓDIGO DE CARGO ACTUAL  300510 non-null  object
 9   FUENTE DE PAGO          300510 non-null  object
 10  SERVICIO                300510 non-null  object
 11  TURNOS                  295703 non-null  object
dtypes: int64(1), object(11)
memory usage: 27.5+ MB


In [11]:
# Limpiar nombres de columnas (convertir a minúsculas y reemplazar espacios con _)
real.columns = real.columns.str.strip().str.lower().str.replace(' ', '_')

# Verificar los nombres de columnas actuales
print("Columnas disponibles:", real.columns.tolist())

# Seleccionar y renombrar columnas según los nombres proporcionados
real = real.rename(columns={
    'id_trabajador': 'id_personal',
    'nombre_completo': 'nombre_completo',
    'curp': 'curp',
    'rfc': 'rfc',
    'clues': 'clues',
    'puesto': 'puesto',
    'estatus': 'estatus',
    'unidad_de_comision': 'unidad_de_comisión',
    'codigo_de_cargo_actual': 'código_de_cargo_actual',
    'fuente_de_pago': 'fuente_de_pago',
    'servicio': 'servicio',  # Si existe esta columna
    'turnos': 'turnos'
})

# Si alguna columna no existe en el original, la creamos vacía
columnas_requeridas = [
    'id_personal', 'nombre_completo', 'curp', 'rfc', 'clues', 'puesto',
    'estatus', 'unidad_de_comisión', 'código_de_cargo_actual',
    'fuente_de_pago', 'servicio', 'turnos'
]

for col in columnas_requeridas:
    if col not in real.columns:
        real[col] = None  # o pd.NA para valores faltantes

# Seleccionar solo las columnas requeridas
real = real[columnas_requeridas]
mapeo_clues = {
    "GRIMB007292": "GRIMB010536",
    "OCIMB003162": "OCIMB009655",
    "OCIMB004685": "OCIMB009626",
    "SRIMB000885": "SRIMB000016",
    "NTMBI000551": "NTIMB000551" # Incluir también el reemplazo que ya tenías
}
real['clues'] = real['clues'].replace(mapeo_clues)

# Limpieza y transformación de datos
real['clues'] = real['clues'].str.strip()
real['código_de_cargo_actual'] = real['código_de_cargo_actual'].str.strip()
# Reemplazar valores específicos
real['clues'] = real['clues'].replace('NTMBI000551', 'NTIMB000551')
real['unidad_de_comisión'] = real['unidad_de_comisión'].replace('SIN_CODIGO', 'SIN_COMISIÓN')
# Reemplazar valores específicos
real['clues'] = real['clues'].replace('NTMBI000551', 'NTIMB000551')
real['unidad_de_comisión'] = real['unidad_de_comisión'].replace('SIN_CODIGO', 'SIN_COMISIÓN')

# Condición para SIN_COMISIÓN donde clues == unidad_de_comisión
real.loc[real['clues'] == real['unidad_de_comisión'], 'unidad_de_comisión'] = 'SIN_COMISIÓN'

# Actualizar clues donde hay unidad_de_comisión válida
mask = (real['unidad_de_comisión'].notna()) & (real['unidad_de_comisión'] != 'SIN_COMISIÓN')
real.loc[mask, 'clues'] = real['unidad_de_comisión']

# Limpiar espacios nuevamente
real['clues'] = real['clues'].str.strip()

# Filtrar registros no deseados
real = real[~real['estatus'].isin(['BAJA', 'PREJUBILACION'])]

# Crear columna estatus_cargo
real['estatus_cargo'] = 0
real.loc[(real['código_de_cargo_actual'].notna()) &
         (real['código_de_cargo_actual'] != 'SIN_PUESTO'), 'estatus_cargo'] = 1

# Mostrar esultados
print("\nForma del DataFrame:", real.shape)
print("\nPrimeras filas:")
print(real.head())
print("\nInformación del DataFrame:")
print(real.info()) # Crear el equivalente a real_cnpm
real_cnpm = real.copy()


# real_cnpm = real_cnpm[real_cnpm['clues'].isin(vec_ideal)]

# Agrupar y resumir
real_cnpm = real_cnpm.groupby(['clues', 'código_de_cargo_actual', 'estatus_cargo'], as_index=False).agg(
    total_real=('id_personal', 'count'),
    pago_imb=('fuente_de_pago', lambda x: (x == "IMSS BIENESTAR").sum())
)

# Renombrar columnas
real_cnpm = real_cnpm.rename(columns={
    'clues': 'clues_imb',
    'código_de_cargo_actual': 'codigo_cnpm',
    'estatus_cargo': 'estatus_puesto'
})

# Seleccionar columnas en el orden deseado
real_cnpm = real_cnpm[['clues_imb', 'codigo_cnpm', 'total_real', 'pago_imb', 'estatus_puesto']]

# Mostrar resultados
print("\nDataFrame real_cnpm:")
print(real_cnpm.head())
print("\nInformación:")
print(real_cnpm.info())
print(real.columns.tolist())   # 1. Leer claves de entidades
clave_ent = pd.read_csv("/content/drive/MyDrive/imms bienestar/claves_entidades.csv")[['clave_entidad', 'entidad']]

# 2. Leer catálogo de unidades CLUES
clues_febrero = pd.read_csv("/content/drive/MyDrive/imms bienestar/clues_marzo.csv")[['clues_imb', 'nombre_de_la_unidad', 'entidad', 'latitud', 'longitud']]

# 3. Leer catálogo de cargos CNPM
# Primero limpiamos los nombres de las columnas (similar a clean_names() de R)
catalogo_cargo = pd.read_csv("/content/drive/MyDrive/imms bienestar/Catálogo Cargo Actual CNPM 20250414.csv")

# Limpiar nombres de columnas (convertir a minúsculas y reemplazar espacios/acentos)
catalogo_cargo.columns = (
    catalogo_cargo.columns
    .str.lower()  # Convertir a minúsculas
    .str.normalize('NFKD')  # Separar caracteres acentuados
    .str.encode('ascii', errors='ignore')  # Eliminar caracteres no ASCII
    .str.decode('utf-8')  # Volver a cadena UTF-8
    .str.replace(' ', '_')  # Reemplazar espacios con guiones bajos
    .str.replace('[^a-z0-9_]', '', regex=True)  # Eliminar caracteres especiales
)

# Seleccionar columnas requeridas
catalogo_cargo = catalogo_cargo[['codigo_cnpm', 'denominacion_del_puesto']]

# Verificación de los datos cargados
print("=== Claves de entidades ===")
print(clave_ent.head())
print("\n=== CLUES marzo ===")
print(clues_febrero.head())
print("\n=== Catálogo de cargos ===")
print(catalogo_cargo.head())  # Realizar el full join (merge outer)
brecha = pd.merge(
    ideal_cnpm,
    real_cnpm,
    on=['clues_imb', 'codigo_cnpm'],
    how='outer'
)

# Reemplazar valores NA
brecha['total_ideal'] = brecha['total_ideal'].fillna(0)
brecha['total_real'] = brecha['total_real'].fillna(0)
brecha['pago_imb'] = brecha['pago_imb'].fillna(0)

# Calcular brecha y excedente
brecha['brecha'] = np.where(
    brecha['total_ideal'] >= brecha['total_real'],
    brecha['total_ideal'] - brecha['total_real'],
    0
)

brecha['excedente'] = np.where(
    brecha['total_ideal'] < brecha['total_real'],
    brecha['total_real'] - brecha['total_ideal'],
    0
)

# Unir con clues_febrero
brecha = pd.merge(
    brecha,
    clues_febrero,
    on='clues_imb',
    how='left'
)

# Unir con catalogo_cargo
brecha = pd.merge(
    brecha,
    catalogo_cargo,
    on='codigo_cnpm',
    how='left'
)

# Manejar estatus_puesto
brecha['estatus_puesto'] = brecha['estatus_puesto'].astype(str)
brecha['estatus_puesto'] = brecha['estatus_puesto'].fillna('solo en ideal')

# Asignaciones especiales para entidad
condiciones_entidad = [
    brecha['clues_imb'] == "Arantepacua",
    brecha['clues_imb'] == "Atenco",
    brecha['clues_imb'] == "MSIMBJIUT",
    brecha['clues_imb'].isin(["OCSSA007705", "OCSSA021396"]),
    brecha['clues_imb'] == "VZIMB008946",
    brecha['clues_imb'] == "YNSSA000565"
]

opciones_entidad = [
    "MICHOACAN DE OCAMPO",
    "MEXICO",
    "MORELOS",
    "OAXACA",
    "VERACRUZ DE IGNACIO DE LA LLAVE",
    "YUCATAN"
]

brecha['entidad'] = np.select(condiciones_entidad, opciones_entidad, default=brecha['entidad'])

# Asignaciones especiales para nombre_de_la_unidad
condiciones_unidad = [
    brecha['clues_imb'] == "Arantepacua",
    brecha['clues_imb'] == "Atenco",
    brecha['clues_imb'] == "MSIMBJIUT",
    brecha['clues_imb'] == "OCSSA007705",
    brecha['clues_imb'] == "OCSSA021396",
    brecha['clues_imb'] == "YNSSA000565"
]

opciones_unidad = [
    "UNIDAD NUEVA ARANTEPACUA",
    "UNIDAD NUEVA ATENCO",
    "UNIDAD NUEVA MSIMBJIUT",
    "UNIDAD NUEVA OAXACA 1",
    "UNIDAD NUEVA OAXACA 2",
    "UNIDAD NUEVA EN YUCATAN"
]

brecha['nombre_de_la_unidad'] = np.select(condiciones_unidad, opciones_unidad, default=brecha['nombre_de_la_unidad'])

# Unir con clave_ent
brecha = pd.merge(
    brecha,
    clave_ent,
    on='entidad',
    how='left'
)

# Seleccionar columnas finales
columnas_finales = [
    'clave_entidad', 'entidad', 'clues_imb', 'nombre_de_la_unidad',
    'codigo_cnpm', 'denominacion_del_puesto', 'total_ideal',
    'total_real', 'pago_imb', 'brecha', 'excedente',
    'estatus_puesto', 'latitud', 'longitud'
]

brecha = brecha[columnas_finales]

# Guardar como Excel

os.makedirs("/content/Brecha_nacional/", exist_ok=True)

# Guardar como Excel
brecha.to_excel("/content/Brecha_nacional/brecha_aritmetica_min_29_05_25.xlsx", index=False)

print("Proceso completado. Archivo guardado en /content/Brecha_nacional/")
print(f"Dimensiones del DataFrame: {brecha.shape}")
print("\nPrimeras filas:")
print(brecha.head())  # Crear la columna clasificacion_cargo
brecha['clasificacion_cargo'] = np.select(
    [
        brecha['codigo_cnpm'].str.contains('CG', na=False),
        brecha['codigo_cnpm'].str.contains('EN', na=False),
        brecha['codigo_cnpm'].str.contains('ME', na=False),
        brecha['codigo_cnpm'].str.contains('MG', na=False),
        brecha['codigo_cnpm'].str.contains('OP', na=False),
        brecha['codigo_cnpm'].str.contains('FA', na=False),
        brecha['codigo_cnpm'] == 'SIN_PUESTO'
    ],
    [
        'Cuerpos de gobierno',
        'Enfermería',
        'Médicos especialistas',
        'Médicos generales',
        'Personal operativo',
        'Tradicional',
        'Sin cargo'
    ],
    default=None
)

# Seleccionar solo columnas numéricas para el resumen
numeric_cols = brecha.select_dtypes(include=[np.number]).columns.tolist()

# Crear el resumen agrupado
brecha_resumen = brecha.groupby('clasificacion_cargo', as_index=False)[numeric_cols].sum()

# Ordenar las columnas (opcional)
column_order = ['clasificacion_cargo'] + numeric_cols
brecha_resumen = brecha_resumen[column_order]

# Crear el directorio si no existe
os.makedirs("/content/Brecha_nacional/Resumen_de_brecha/", exist_ok=True)

# Guardar como Excel
brecha_resumen.to_excel("/content/Brecha_nacional/Resumen_de_brecha/brecha_clasificacion_min_14_04_25.xlsx", index=False)

print("Resumen por clasificación guardado correctamente")
print("\nTabla resumen:")
print(brecha_resumen)  # 1. Crear clasificación de cargos (similar al paso anterior)
conditions = [
    brecha['codigo_cnpm'].str.contains('CG', na=False),
    brecha['codigo_cnpm'].str.contains('EN', na=False),
    brecha['codigo_cnpm'].str.contains('ME', na=False),
    brecha['codigo_cnpm'].str.contains('MG', na=False),
    brecha['codigo_cnpm'].str.contains('OP', na=False),
    brecha['codigo_cnpm'].str.contains('FA', na=False),
    brecha['codigo_cnpm'].isin(['SIN_PUESTO'])
]

choices = [
    'Cuerpos de gobierno',
    'Enfermería',
    'Médicos especialistas',
    'Médicos generales',
    'Personal operativo',
    'Tradicional',
    'Sin cargo'
]

brecha['clasificacion_cargo'] = np.select(conditions, choices, default=None)

# 2. Filtrar y agrupar por entidad y clasificación
brecha_por_entidad = brecha[~brecha['clasificacion_cargo'].isna()]
brecha_por_entidad = brecha_por_entidad[~brecha_por_entidad['entidad'].isin(['YUCATAN', 'GUANAJUATO'])]

numeric_cols = brecha_por_entidad.select_dtypes(include=[np.number]).columns.tolist()
brecha_por_entidad = brecha_por_entidad.groupby(['entidad', 'clasificacion_cargo'], as_index=False)[numeric_cols].sum()

# 3. Crear tablas pivotadas
entidad_clas_brecha = brecha_por_entidad.pivot_table(
    index='entidad',
    columns='clasificacion_cargo',
    values='brecha',
    aggfunc='sum',
    fill_value=0
).reset_index()

entidad_clas_excedente = brecha_por_entidad.pivot_table(
    index='entidad',
    columns='clasificacion_cargo',
    values='excedente',
    aggfunc='sum',
    fill_value=0
).reset_index()

# 4. Guardar en un archivo Excel con múltiples hojas
from pandas import ExcelWriter

# Crear directorio si no existe
os.makedirs("/content/Brecha_nacional/Resumen_de_brecha/", exist_ok=True)

with ExcelWriter("/content/Brecha_nacional/Resumen_de_brecha/brecha_entidad_clasificacion_min.xlsx") as writer:
    entidad_clas_brecha.to_excel(writer, sheet_name='brecha', index=False)
    entidad_clas_excedente.to_excel(writer, sheet_name='excedente', index=False)

print("Tablas por entidad guardadas correctamente")
print("\nBrecha por entidad y clasificación:")
print(entidad_clas_brecha.head())
print("\nExcedente por entidad y clasificación:")
print(entidad_clas_excedente.head())  # Crear directorio de descargas si no existe
downloads_dir = "/content/Descargas_Brechas/"
os.makedirs(downloads_dir, exist_ok=True)

# Función para filtrar y guardar brechas
def guardar_brecha(df, filtro, nombre_archivo):
    """
    Filtra el dataframe y guarda el resultado en un archivo Excel

    Args:
    df: DataFrame principal con todos los datos
    filtro: Condición de filtrado (ej: df['entidad'] == "MICHOACAN DE OCAMPO")
    nombre_archivo: Nombre del archivo de salida
    """
    df_filtrado = df[filtro]

    # Eliminar filas donde total_ideal y total_real son ambos 0 (excepto para algunos casos especiales)
    # Esta condición parece aplicarse solo si "brecha_" NO está en el nombre del archivo.
    # El código R original aplica este filtro de manera más amplia. Mantendremos la
    # lógica original de Python tal como está escrita, asumiendo que es intencional.
    if "brecha_" not in nombre_archivo:
         df_filtrado = df_filtrado[~((df_filtrado['total_ideal'] == 0) & (df_filtrado['total_real'] == 0))]


    # Guardar archivo
    ruta_completa = f"{downloads_dir}{nombre_archivo}"
    df_filtrado.to_excel(ruta_completa, index=False)
    print(f"Archivo guardado: {ruta_completa}")

# 1. Brecha para Michoacán
guardar_brecha(brecha, brecha['entidad'] == "MICHOACAN DE OCAMPO", "brecha_min_MICHOACAN.xlsx")

# 2. Brechas para CLUES específicas
clues_lista = [
    "YNIMB000024", "YNIMB000012", "YNIMB000036",
    "TCIMB000791", "TCIMB000786", "TCIMB000774",
    "TCIMB000803", "TCIMB000815", "DFIMB002341",
    "DFIMB002114", "SPIMB000585", "GRIMB010536",
    "VZIMB008946", "SRIMB000016", "BCIMB000623",
    "SRIMB000050", "OCIMB008260", "CSIMB001732"
]

for clues in clues_lista:
    nombre_archivo = f"brecha_min_{clues}.xlsx" if clues != "GRIMB010536" else f"{clues}_min_brecha.xlsx"
    guardar_brecha(brecha, brecha['clues_imb'] == clues, nombre_archivo)

# 3. Brechas para estados completos
estados = {
    "TABASCO": "brecha_min_tabasco.xlsx",
    "VERACRUZ DE IGNACIO DE LA LLAVE": "brecha_min_veracruz.xlsx"
}

for estado, archivo in estados.items():
    guardar_brecha(brecha, brecha['entidad'] == estado, archivo)

print("\nProceso completado. Todos los archivos se han guardado en:", downloads_dir)  # 1. Filtrar para CSIMB003511
CSIMB003511_brecha = brecha[
    (brecha['clues_imb'] == "CSIMB003511") &
    (brecha['total_ideal'] != 0) &
    (brecha['total_real'] != 0)
]

# 2. Procesar CSIMB006533
CSIMB006533_brecha = brecha[
    (brecha['clues_imb'] == "CSIMB006533") &
    (brecha['total_ideal'] != 0) &
    (brecha['total_real'] != 0)
]

# Clasificación por grupos para CSIMB006533
conditions = [
    CSIMB006533_brecha['codigo_cnpm'].str.contains('CG', na=False),
    CSIMB006533_brecha['codigo_cnpm'].str.contains('EN', na=False),
    CSIMB006533_brecha['codigo_cnpm'].str.contains('ME', na=False),
    CSIMB006533_brecha['codigo_cnpm'].str.contains('MG', na=False),
    CSIMB006533_brecha['codigo_cnpm'].str.contains('OP', na=False),
    CSIMB006533_brecha['codigo_cnpm'].str.contains('FA', na=False),
    CSIMB006533_brecha['codigo_cnpm'] == "SIN_PUESTO"
]

choices = [
    "Cuerpos de gobierno",
    "Enfermería",
    "Médicos especialistas",
    "Médicos generales",
    "Personal operativo",
    "Tradicional",
    "Sin cargo"
]

CSIMB006533_brecha['clasificacion_cargo'] = np.select(conditions, choices, default=None)

# Resumen por grupos
CSIMB006533_brecha_grupos = CSIMB006533_brecha.groupby('clasificacion_cargo', as_index=False).agg({
    'total_ideal': 'sum',
    'total_real': 'sum',
    'brecha': 'sum',
    'excedente': 'sum'
}).rename(columns={
    'clasificacion_cargo': 'Clasificación del Cargo',
    'total_ideal': 'Total Ideal',
    'total_real': 'Total Ocupado',
    'brecha': 'Brecha',
    'excedente': 'Excedente'
})

# Filtrar médicos especialistas para CSIMB006533
CSIMB006533_brecha_me = CSIMB006533_brecha[
    CSIMB006533_brecha['codigo_cnpm'].str.contains('ME', na=False)
][['codigo_cnpm', 'denominacion_del_puesto', 'total_ideal', 'total_real', 'brecha', 'excedente']].rename(columns={
    'codigo_cnpm': 'Código de cargo actual',
    'denominacion_del_puesto': 'cargo',
    'total_ideal': 'Total Ideal',
    'total_real': 'Total Ocupado',
    'brecha': 'Brecha',
    'excedente': 'Excedente'
})

# 3. Procesar MCIMB011986
MCIMB011986_brecha = brecha[
    (brecha['clues_imb'] == "MCIMB011986") &
    (brecha['codigo_cnpm'].isin(["ME003", "ME001", "ME005", "ME071"]))
]

# 4. Procesar MCIMB011945
MCIMB011945_brecha = brecha[
    (brecha['clues_imb'] == "MCIMB011945") &
    (brecha['total_ideal'] != 0) &
    (brecha['total_real'] != 0)
]

# Mostrar resultados (equivalente a write_clip en R)
print("=== CSIMB006533 por Grupos ===")
print(CSIMB006533_brecha_grupos.to_markdown(index=False))

print("\n=== CSIMB006533 Médicos Especialistas ===")
print(CSIMB006533_brecha_me.to_markdown(index=False))

print("\n=== MCIMB011986 ===")
print(MCIMB011986_brecha.to_markdown(index=False))

print("\n=== MCIMB011945 ===")
print(MCIMB011945_brecha.to_markdown(index=False))

# Opcional: Guardar en archivos Excel
CSIMB006533_brecha_grupos.to_excel("CSIMB006533_grupos.xlsx", index=False)
CSIMB006533_brecha_me.to_excel("CSIMB006533_medicos_especialistas.xlsx", index=False)
MCIMB011986_brecha.to_excel("MCIMB011986.xlsx", index=False)
MCIMB011945_brecha.to_excel("MCIMB011945.xlsx", index=False)           # Diccionario de turnos válidos (equivalente a la lista nombrada de R)
turnos_validos = {
    "Matutino": ["lunes-matutino", "martes-matutino", "miercoles-matutino", "jueves-matutino", "viernes-matutino"],
    "Matutino B": ["miercoles-matutino", "jueves-matutino", "viernes-matutino", "sabado-matutino", "domingo-matutino"],
    "Vespertino": ["lunes-vespertino", "martes-vespertino", "miercoles-vespertino", "jueves-vespertino", "viernes-vespertino"],
    "Nocturno A": ["lunes-nocturno", "miercoles-nocturno", "viernes-nocturno"],
    "Nocturno B": ["martes-nocturno", "jueves-nocturno", "sabado-nocturno"],
    "Jornada acumulada": ["sabado-jornadaacumulada", "domingo-jornadaacumulada"]
}

# Función para verificar si un turno es válido
def es_turno_valido(turno):
    """Verifica si un turno está en alguno de los grupos válidos"""
    return any(turno in turnos for turnos in turnos_validos.values())

# Ejemplo de uso:
# Para filtrar un DataFrame por turnos válidos:
# df_turnos_validos = df[df['turno'].apply(es_turno_valido)]

# También puedes crear una columna con la categoría de turno:
def categorizar_turno(turno):
    """Devuelve la categoría a la que pertenece un turno"""
    for categoria, turnos in turnos_validos.items():
        if turno in turnos:
            return categoria
    return "Otro"

# Ejemplo:
# df['categoria_turno'] = df['turno'].apply(categorizar_turno)  # Definimos la función para asignar turnos (equivalente a asignar_turno en R)
def asignar_turno(turno_str):
    """Asigna la categoría de turno basado en coincidencia exacta con turnos_validos"""
    if pd.isna(turno_str):
        return "Otro"

    # Normalizar texto (equivalente a stri_trans_general)
    turno_str = unicodedata.normalize('NFKD', turno_str).encode('ascii', 'ignore').decode('ascii')

    # Separar los turnos individuales
    turnos_persona = [t.strip() for t in turno_str.split(',')]

    # Buscar coincidencia exacta con los grupos definidos
    for categoria, turnos in turnos_validos.items():
        if set(turnos) == set(turnos_persona):
            return categoria

    return "Otro"

# Procesamiento equivalente al pipeline de R
ocupado_por_turnos = (
    real
    .assign(
        # Eliminar espacios en blanco
        # Cambiar 'turno' a 'turnos' aquí
        turnos=lambda x: x['turnos'].str.replace(r'\s', '', regex=True),
        # Aplicar la función de asignación de turnos
        # Aplicar la función a la columna 'turnos'
        turno_real=lambda x: x['turnos'].apply(asignar_turno)
    )
    # Seleccionar columnas y renombrar clues
    # Usa 'código_de_cargo_actual' en lugar de 'codigo_cnpm'
    [['clues', 'código_de_cargo_actual', 'turno_real']]
    .rename(columns={'clues': 'clues_imb', 'código_de_cargo_actual': 'codigo_cnpm'})
    # Agrupar y contar
    .groupby(['clues_imb', 'codigo_cnpm', 'turno_real'], as_index=False)
    .size()
    .rename(columns={'size': 'total_real'})
    # Pivotar a formato wide
    .pivot_table(
        index=['clues_imb', 'codigo_cnpm'],
        columns='turno_real',
        values='total_real',
        fill_value=0
    )
    .reset_index()
    # Seleccionar y ordenar columnas
    [['clues_imb', 'codigo_cnpm', 'Matutino', 'Matutino B', 'Vespertino',
      'Nocturno A', 'Nocturno B', 'Jornada acumulada', 'Otro']]
)

# Mostrar resultado
print("Resumen de ocupación por turnos:")
print(ocupado_por_turnos.head())

Columnas disponibles: ['id_trabajador', 'nombre_completo', 'curp', 'rfc', 'clues', 'puesto', 'estatus', 'unidad_de_comisión', 'código_de_cargo_actual', 'fuente_de_pago', 'servicio', 'turnos']


<ipython-input-11-9bbe5427800e>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real['estatus_cargo'] = 0



Forma del DataFrame: (299971, 13)

Primeras filas:
   id_personal                   nombre_completo                curp  \
0            1          ABAD DIAZ NELIDA NEREYDA  AADN980711MGRBZL03   
1            2      ABAD ESPINOZA NADIA ANGELICA  AAEN940825MPLBSD06   
2            3            ABAD VARGAS DESCIDERIO  AAVD690610HGRBRS06   
3            4     ABADIA AGUILAR FRANCISCO IVAN  AAAF940319HCSBGR06   
4            5  ABADIA GORDILLO BEATRIZ NICOLAZA  AAGB800910MCSBRT07   

             rfc         clues  puesto estatus unidad_de_comisión  \
0  AADN980711650   GRIMB009042  M12001  ACTIVO       SIN_COMISIÓN   
1  AAEN940825ME5  SIN_COMISION  M12003  ACTIVO       SIN_COMISION   
2  AAVD690610U92  SIN_COMISION  M01006  ACTIVO       SIN_COMISION   
3  AAAF9403197W1  SIN_COMISION  M12013  ACTIVO       SIN_COMISION   
4  AAGB8009106S5  SIN_COMISION  M02035  ACTIVO       SIN_COMISION   

  código_de_cargo_actual  fuente_de_pago             servicio  \
0                  EN005  IMSS BIEN

<ipython-input-11-9bbe5427800e>:443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CSIMB006533_brecha['clasificacion_cargo'] = np.select(conditions, choices, default=None)


Resumen de ocupación por turnos:
turno_real    clues_imb codigo_cnpm  Matutino  Matutino B  Vespertino  \
0           BCIMB000010       CG001       1.0         0.0         0.0   
1           BCIMB000010       CG028       1.0         0.0         0.0   
2           BCIMB000010       EN001       3.0         0.0         2.0   
3           BCIMB000010       EN002       0.0         0.0         0.0   
4           BCIMB000010       EN017       1.0         0.0         0.0   

turno_real  Nocturno A  Nocturno B  Jornada acumulada  Otro  
0                  0.0         0.0                0.0   0.0  
1                  0.0         0.0                0.0   0.0  
2                  2.0         0.0                1.0   1.0  
3                  0.0         0.0                1.0   1.0  
4                  0.0         0.0                0.0   0.0  


In [12]:
# Limpiar nombres de columnas (convertir a minúsculas y reemplazar espacios con _)
real.columns = real.columns.str.strip().str.lower().str.replace(' ', '_')

# Verificar los nombres de columnas actuales
print("Columnas disponibles:", real.columns.tolist())

# Seleccionar y renombrar columnas según los nombres proporcionados
real = real.rename(columns={
    'id_trabajador': 'id_personal',
    'nombre_completo': 'nombre_completo',
    'curp': 'curp',
    'rfc': 'rfc',
    'clues': 'clues',
    'puesto': 'puesto',
    'estatus': 'estatus',
    'unidad_de_comision': 'unidad_de_comisión',
    'codigo_de_cargo_actual': 'código_de_cargo_actual',
    'fuente_de_pago': 'fuente_de_pago',
    'servicio': 'servicio',  # Si existe esta columna
    'turnos': 'turnos'
})

# Si alguna columna no existe en el original, la creamos vacía
columnas_requeridas = [
    'id_personal', 'nombre_completo', 'curp', 'rfc', 'clues', 'puesto',
    'estatus', 'unidad_de_comisión', 'código_de_cargo_actual',
    'fuente_de_pago', 'servicio', 'turnos'
]

for col in columnas_requeridas:
    if col not in real.columns:
        real[col] = None  # o pd.NA para valores faltantes

# Seleccionar solo las columnas requeridas
real = real[columnas_requeridas]

# Limpieza y transformación de datos
real['clues'] = real['clues'].str.strip()
real['código_de_cargo_actual'] = real['código_de_cargo_actual'].str.strip()

# Reemplazar valores específicos
real['clues'] = real['clues'].replace('NTMBI000551', 'NTIMB000551')
real['unidad_de_comisión'] = real['unidad_de_comisión'].replace('SIN_CODIGO', 'SIN_COMISIÓN')

# Condición para SIN_COMISIÓN donde clues == unidad_de_comisión
real.loc[real['clues'] == real['unidad_de_comisión'], 'unidad_de_comisión'] = 'SIN_COMISIÓN'

# Actualizar clues donde hay unidad_de_comisión válida
mask = (real['unidad_de_comisión'].notna()) & (real['unidad_de_comisión'] != 'SIN_COMISIÓN')
real.loc[mask, 'clues'] = real['unidad_de_comisión']

# Limpiar espacios nuevamente
real['clues'] = real['clues'].str.strip()

# Filtrar registros no deseados
real = real[~real['estatus'].isin(['BAJA', 'PREJUBILACION'])]

# Crear columna estatus_cargo
real['estatus_cargo'] = 0
real.loc[(real['código_de_cargo_actual'].notna()) &
         (real['código_de_cargo_actual'] != 'SIN_PUESTO'), 'estatus_cargo'] = 1

# Mostrar resultados
print("\nForma del DataFrame:", real.shape)
print("\nPrimeras filas:")
print(real.head())
print("\nInformación del DataFrame:")
print(real.info()) # Crear el equivalente a real_cnpm
real_cnpm = real.copy()

# Filtrar (opcional, comentado como en R)
# real_cnpm = real_cnpm[real_cnpm['clues'].isin(vec_ideal)]

# Agrupar y resumir
real_cnpm = real_cnpm.groupby(['clues', 'código_de_cargo_actual', 'estatus_cargo'], as_index=False).agg(
    total_real=('id_personal', 'count'),
    pago_imb=('fuente_de_pago', lambda x: (x == "IMSS BIENESTAR").sum())
)

# Renombrar columnas
real_cnpm = real_cnpm.rename(columns={
    'clues': 'clues_imb',
    'código_de_cargo_actual': 'codigo_cnpm',
    'estatus_cargo': 'estatus_puesto'
})

# Seleccionar columnas en el orden deseado
real_cnpm = real_cnpm[['clues_imb', 'codigo_cnpm', 'total_real', 'pago_imb', 'estatus_puesto']]

# Mostrar resultados
print("\nDataFrame real_cnpm:")
print(real_cnpm.head())
print("\nInformación:")
print(real_cnpm.info())
print(real.columns.tolist())   # 1. Leer claves de entidades
# Actualizar la ruta para reflejar la ubicación del archivo en Google Drive
clave_ent = pd.read_csv("/content/drive/MyDrive/imms bienestar/claves_entidades.csv")[['clave_entidad', 'entidad']] # Asumiendo que el archivo está en la carpeta 'imms bienestar'

# 2. Leer catálogo de unidades CLUES
# Se eliminan las columnas 'latitud' y 'longitud'
clues_febrero = pd.read_csv("/content/drive/MyDrive/imms bienestar/clues_marzo.csv")[['clues_imb', 'nombre_de_la_unidad', 'entidad']]

# 3. Leer catálogo de cargos CNPM
# Primero limpiamos los nombres de las columnas (similar a clean_names() de R)
catalogo_cargo = pd.read_csv("/content/drive/MyDrive/imms bienestar/Catálogo Cargo Actual CNPM 20250414.csv") # Actualizar ruta si este archivo también está en Drive

# Limpiar nombres de columnas (convertir a minúsculas y reemplazar espacios/acentos)
catalogo_cargo.columns = (
    catalogo_cargo.columns
    .str.lower()  # Convertir a minúsculas
    .str.normalize('NFKD')  # Separar caracteres acentuados
    .str.encode('ascii', errors='ignore')  # Eliminar caracteres no ASCII
    .str.decode('utf-8')  # Volver a cadena UTF-8
    .str.replace(' ', '_')  # Reemplazar espacios con guiones bajos
    .str.replace('[^a-z0-9_]', '', regex=True)  # Eliminar caracteres especiales
)

# Seleccionar columnas requeridas
catalogo_cargo = catalogo_cargo[['codigo_cnpm', 'denominacion_del_puesto']]

# Verificación de los datos cargados
print("=== Claves de entidades ===")
print(clave_ent.head())
print("\n=== CLUES marzo ===")
print(clues_febrero.head())
print("\n=== Catálogo de cargos ===")
print(catalogo_cargo.head())

Columnas disponibles: ['id_personal', 'nombre_completo', 'curp', 'rfc', 'clues', 'puesto', 'estatus', 'unidad_de_comisión', 'código_de_cargo_actual', 'fuente_de_pago', 'servicio', 'turnos', 'estatus_cargo']

Forma del DataFrame: (299971, 13)

Primeras filas:
   id_personal                   nombre_completo                curp  \
0            1          ABAD DIAZ NELIDA NEREYDA  AADN980711MGRBZL03   
1            2      ABAD ESPINOZA NADIA ANGELICA  AAEN940825MPLBSD06   
2            3            ABAD VARGAS DESCIDERIO  AAVD690610HGRBRS06   
3            4     ABADIA AGUILAR FRANCISCO IVAN  AAAF940319HCSBGR06   
4            5  ABADIA GORDILLO BEATRIZ NICOLAZA  AAGB800910MCSBRT07   

             rfc         clues  puesto estatus unidad_de_comisión  \
0  AADN980711650   GRIMB009042  M12001  ACTIVO       SIN_COMISIÓN   
1  AAEN940825ME5  SIN_COMISION  M12003  ACTIVO       SIN_COMISIÓN   
2  AAVD690610U92  SIN_COMISION  M01006  ACTIVO       SIN_COMISIÓN   
3  AAAF9403197W1  SIN_COMISION  

In [16]:
def consultar_por_clues():
    """Solicita un CLUES y muestra su reporte detallado"""
    # Mostrar lista de CLUES disponibles para referencia
    print("\nCLUES disponibles ():")
    print(brecha['clues_imb'].unique()[:10])

    # Solicitar input
    clues_input = input("\n🔎 Ingresa el CLUES a consultar (ej: BCIMB000010): ").strip().upper()

    # Filtrar datos
    df_clues = brecha[brecha['clues_imb'] == clues_input]

    if df_clues.empty:
        print(f"\n❌ No se encontró el CLUES: {clues_input}")
        return

    # Obtener metadatos
    unidad = df_clues['nombre_de_la_unidad'].iloc[0]
    entidad = df_clues['entidad'].iloc[0]

    # 1. Resumen general
    print(f"\n📋 REPORTE PARA: {unidad} ({clues_input})")
    print(f"📍 Entidad: {entidad}")
    print(f"📅 Fecha de análisis: {pd.Timestamp.now().strftime('%d/%m/%Y')}")
    print("\n" + "═"*50)

    # 2. Brecha por clasificación
    print("\n🔍 DISTRIBUCIÓN POR CLASIFICACIÓN DE CARGO")
    resumen_clasificacion = df_clues.groupby('clasificacion_cargo').agg(
        Ideal=('total_ideal', 'sum'),
        Ocupado=('total_real', 'sum'),
        Brecha=('brecha', 'sum'),
        Excedente=('excedente', 'sum')
    ).reset_index()
    print(resumen_clasificacion.to_markdown(index=False))

    # 3. Top 5 brechas
    print("\n📉 TOP 5 PUESTOS CON MAYOR BRECHA")
    top_brecha = df_clues.nlargest(5, 'brecha')[[
        'codigo_cnpm',
        'denominacion_del_puesto',
        'total_ideal',
        'total_real',
        'brecha'
    ]]
    print(top_brecha.to_markdown(index=False))

    # 4. Top 5 excedentes
    print("\n📈 TOP 5 PUESTOS CON MAYOR EXCEDENTE")
    top_excedente = df_clues.nlargest(5, 'excedente')[[
        'codigo_cnpm',
        'denominacion_del_puesto',
        'total_ideal',
        'total_real',
        'excedente'
    ]]
    print(top_excedente.to_markdown(index=False))

    # 5. Turnos (si existe información)
    if clues_input in ocupado_por_turnos['clues_imb'].values:
        print("\n⏰ DISTRIBUCIÓN DE TURNOS")
        turnos_clues = ocupado_por_turnos[ocupado_por_turnos['clues_imb'] == clues_input]
        print(turnos_clues.drop(columns='clues_imb').to_markdown(index=False))

    # 6. Exportar opción
    exportar = input("\n¿Exportar a Excel? (s/n): ").lower()
    if exportar == 's':
        nombre_archivo = f"reporte_{clues_input}.xlsx"
        with pd.ExcelWriter(nombre_archivo) as writer:
            df_clues.to_excel(writer, sheet_name='Detalle', index=False)
            resumen_clasificacion.to_excel(writer, sheet_name='Resumen', index=False)
        print(f"✅ Reporte exportado como: {nombre_archivo}")

# ejecutar
consultar_por_clues()


CLUES disponibles ():
['BCIMB000010' 'BCIMB000022' 'BCIMB000034' 'BCIMB000046' 'BCIMB000051'
 'BCIMB000063' 'BCIMB000075' 'BCIMB000080' 'BCIMB000092' 'BCIMB000104']

🔎 Ingresa el CLUES a consultar (ej: BCIMB000010): z

❌ No se encontró el CLUES: Z


In [15]:
from dash import Dash, html, dcc, Input, Output, State, dash_table
from jupyter_dash import JupyterDash
import pandas as pd
from io import BytesIO
import datetime

if 'brecha' not in globals():
    print("Error: DataFrame 'brecha' no encontrado.")
else:
    clues_list = sorted(brecha['clues_imb'].unique())
    app = JupyterDash(__name__)

    app.layout = html.Div([
        html.H1("Consulta de Brecha por CLUES", style={'textAlign': 'center'}),

        html.Div([
            dcc.Dropdown(
                id='clues-dropdown',
                options=[{'label': f"{clues} - {brecha[brecha['clues_imb']==clues]['nombre_de_la_unidad'].iloc[0]}",
                          'value': clues} for clues in clues_list],
                placeholder="Selecciona un CLUES",
                style={'width': '100%'}
            ),
        ], style={'padding': '20px'}),

        html.Div(id='output-container'),

        html.Div([
            html.Button("Descargar Excel", id='btn-download', style={'margin': '20px'}),
            dcc.Download(id="download-dataframe-xlsx")
        ], style={'textAlign': 'center'})
    ])

    @app.callback(
        Output('output-container', 'children'),
        Input('clues-dropdown', 'value')
    )
    def update_output(clues_selected):
        if not clues_selected:
            return html.Div("Selecciona un CLUES para ver los resultados",
                            style={'textAlign': 'center', 'padding': '20px'})

        df_clues = brecha[brecha['clues_imb'] == clues_selected]

        if df_clues.empty:
            return html.Div(f"No se encontró información para CLUES: {clues_selected}")

        unidad = df_clues['nombre_de_la_unidad'].iloc[0]
        entidad = df_clues['entidad'].iloc[0]
        fecha = datetime.datetime.now().strftime('%d/%m/%Y')

        # Resumen por clasificación
        resumen_clasificacion = df_clues.groupby('clasificacion_cargo').agg(
            Ideal=('total_ideal', 'sum'),
            Ocupado=('total_real', 'sum'),
            Brecha=('brecha', 'sum'),
            Excedente=('excedente', 'sum')
        ).reset_index()

        top_brecha = df_clues.nlargest(5, 'brecha')[[
            'codigo_cnpm', 'denominacion_del_puesto', 'total_ideal', 'total_real', 'brecha'
        ]]

        top_excedente = df_clues.nlargest(5, 'excedente')[[
            'codigo_cnpm', 'denominacion_del_puesto', 'total_ideal', 'total_real', 'excedente'
        ]]

        return html.Div([
            html.H3(f"📋 REPORTE PARA: {unidad} ({clues_selected})", style={'textAlign': 'center'}),
            html.P(f"📍 Entidad: {entidad}"),
            html.P(f"📅 Fecha de análisis: {fecha}"),
            html.Hr(),

            html.H4("🔍 DISTRIBUCIÓN POR CLASIFICACIÓN DE CARGO"),
            dash_table.DataTable(
                data=resumen_clasificacion.to_dict('records'),
                columns=[{'name': i, 'id': i} for i in resumen_clasificacion.columns],
                style_table={'overflowX': 'auto'},
                style_cell={'padding': '5px'},
                page_size=10
            ),

            html.H4("📉 TOP 5 PUESTOS CON MAYOR BRECHA"),
            dash_table.DataTable(
                data=top_brecha.to_dict('records'),
                columns=[{'name': i, 'id': i} for i in top_brecha.columns],
                style_table={'overflowX': 'auto'},
                style_cell={'padding': '5px'},
                page_size=5
            ),

            html.H4("📈 TOP 5 PUESTOS CON MAYOR EXCEDENTE"),
            dash_table.DataTable(
                data=top_excedente.to_dict('records'),
                columns=[{'name': i, 'id': i} for i in top_excedente.columns],
                style_table={'overflowX': 'auto'},
                style_cell={'padding': '5px'},
                page_size=5
            ),
        ])

    @app.callback(
        Output("download-dataframe-xlsx", "data"),
        Input("btn-download", "n_clicks"),
        State('clues-dropdown', 'value'),
        prevent_initial_call=True
    )
    def download_excel(n_clicks, clues_selected):
        if not clues_selected:
            return None

        df_clues = brecha[brecha['clues_imb'] == clues_selected]
        resumen = df_clues.groupby('clasificacion_cargo').agg(
            Ideal=('total_ideal', 'sum'),
            Ocupado=('total_real', 'sum'),
            Brecha=('brecha', 'sum'),
            Excedente=('excedente', 'sum')
        ).reset_index()

        output = BytesIO()
        with pd.ExcelWriter(output, engine='openpyxl') as writer:
            df_clues.to_excel(writer, sheet_name='Detalle', index=False)
            resumen.to_excel(writer, sheet_name='Resumen', index=False)

        return dcc.send_bytes(output.getvalue(), f"reporte_{clues_selected}.xlsx")

    app.run(mode='inline', port=8050)
    app.run(jupyter_mode="external")

/usr/local/lib/python3.11/dist-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

Dash app running on:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>